#### Importing libraries

In [1]:
#pip install ibm-watson  #Installing IBM modules
#pip install moviepy 
#pip install SpeechRecognition
#pip install python-dotenv
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import SpeechToTextV1
import speech_recognition as sr
import moviepy.editor as mp #extracting audio from video
from os import path
import pandas as pd
import json

#### Converts the speech from the provided video file to audio mp3 format and then into text using the IBM Watson Speech-To-Text API. As a result we will have text in JSON format

In [3]:
#video to voice
video_clip = mp.VideoFileClip("video.mp4")
video_clip.audio.write_audiofile("theaudio.mp3")# transfering to audio
#getting authorisation from IBM Watson 
authenticator = IAMAuthenticator(API_KEY) # API_KEY is generated from IBM Watson
speech_to_text = SpeechToTextV1(authenticator=authenticator)
speech_to_text.set_service_url('https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/662d60ba-29f7-428d-98b3-27d12b178763')
#speech_to_text
with open('theaudio.mp3','rb') as audio_file:
    speech_recognition = speech_to_text.recognize(
        audio=audio_file,
        content_type='audio/mp3',model='en-US_NarrowbandModel').get_result() 
#save as JSON
with open('speech_result.json', 'w') as f:
    json.dump(speech_recognition, f)


chunk:   2%|▏         | 54/3392 [00:00<00:06, 522.92it/s, now=None]

MoviePy - Writing audio in theaudio.mp3


MoviePy - Done.


#### Accessing JSON response from the API 

In [4]:
#access to JSON 
with open('speech_result.json') as access_json:
    read_content = json.load(access_json)
read_content

{'result_index': 0,
 'results': [{'final': True,
   'alternatives': [{'transcript': 'hello everyone and welcome to this presentation ',
     'confidence': 1.0}]},
  {'final': True,
   'alternatives': [{'transcript': "I'm doctor Alan Kaplan a primary care physician from the department of family community medicine at university of Toronto and out your account ",
     'confidence': 0.79}]},
  {'final': True,
   'alternatives': [{'transcript': 'hi my cast a registered respiratory therapist and registered pulmonary function technologists from Western Michigan University Homer Stryker M. D. school of medicine in Kalamazoo Michigan ',
     'confidence': 0.87}]},
  {'final': True,
   'alternatives': [{'transcript': 'today we would like to discuss some important aspects of COPD specifically when we should consider COPD our patience what type of medications are available and some factors to consider when selecting an inhaler ',
     'confidence': 0.83}]},
  {'final': True,
   'alternatives': [{'

#### We can see that text is stored in form of nested dictionary and we have to extract transcript content from it.

In [5]:
read_content['results'][0]

{'final': True,
 'alternatives': [{'transcript': 'hello everyone and welcome to this presentation ',
   'confidence': 1.0}]}

#### Processes the API response into a single text result

In [6]:
#accesing transcript in nested dictionary 
def postProcessingText():
    text=read_content['results']
    for content in text:
        alternatives=content['alternatives']
        for results in alternatives:
            transcript=results['transcript']
            print(transcript)
postProcessingText()        

hello everyone and welcome to this presentation 
I'm doctor Alan Kaplan a primary care physician from the department of family community medicine at university of Toronto and out your account 
hi my cast a registered respiratory therapist and registered pulmonary function technologists from Western Michigan University Homer Stryker M. D. school of medicine in Kalamazoo Michigan 
today we would like to discuss some important aspects of COPD specifically when we should consider COPD our patience what type of medications are available and some factors to consider when selecting an inhaler 
Michael did you know that according to results of recent and I H. survey 
all the proportion of smokers who discuss your P. sometimes with a doctor's rose from forty two percent in two thousand nine sixty seven percent two thousand four thirteen 
more than one quarter of adults reported experiencing COB symptoms did not discuss the symptoms with the health care practitioner 
nearly half of US primary Ke

#### As a result we have a text transcript from a video 